In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.pipeline import make_pipeline
from fast_ml.eda import df_info
from fast_ml.model_development import train_valid_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import keras 
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, mean_absolute_error
from keras.callbacks import EarlyStopping
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
import joblib

def preprocess(df):
   
   '''preprocess your fifa dataframe'''

   df_dropped_func = pd.DataFrame()
   url = [idx for idx, col in enumerate(df.columns) if 'url' in col]
   url = [df.columns[idx] for idx in url]
   df = df.drop(columns= url, errors='ignore')

   df[['pace', 'shooting','passing','dribbling','defending','physic']] = df[['pace', 'shooting','passing','dribbling','defending','physic']].fillna(1)

   df['Penaltyist_Score'] = (0.4 * df['mentality_penalties'] + 0.4 * df['shooting'] + 0.1 * df['power_shot_power'] + 0.1 * df['mentality_vision']) / 4
   # df['player_role'] = df['player_positions'].apply(get_player_role)
   
   
   
   df['height_to_weight_ratio'] = df['height_cm'] / df['weight_kg']
   df_dropped_func.append(df[['height_cm', 'weight_kg']])
   
   df.drop(columns=['height_cm', 'weight_kg'],inplace=True, errors='ignore')
   
   df = df.dropna(subset=['wage_eur', 'value_eur'])
   df['value_to_wage_ratio'] = df.value_eur / df.wage_eur
   df_dropped_func.append(df[['value_eur', 'wage_eur']])
   df.drop(columns=['value_eur', 'wage_eur'], inplace=True, errors='ignore')
   
   columns_drop=['short_name', 'long_name', 'potential', 'dob', 'club_team_id', 'club_name',
       'league_name', 'league_level', 'club_loaned_from', 'club_joined','club_contract_valid_until', 'nationality_name',
       'nation_team_id', 'nation_jersey_number','international_reputation','release_clause_eur', 
       'player_tags', 'player_traits', 'nation_position','goalkeeping_speed','club_position']
  
   
   df_dropped_func = df[columns_drop]
   df = df.drop(columns=columns_drop, errors='ignore')
   
   df = df[df['club_jersey_number'].notnull()]

   df['attacking_ratio'] = (df.attacking_crossing + df.attacking_finishing 
                         + df.attacking_heading_accuracy + df.attacking_short_passing + df.attacking_volleys)/5
   df['main_ratio'] = (df.pace + df.shooting + df.passing + df.dribbling + df.defending + df.physic)/6
   df['skill_ratio'] = (df.skill_dribbling + df.skill_curve + df.skill_fk_accuracy + df.skill_long_passing + df.skill_ball_control) / 5
   df['movement_ratio'] = (df.movement_acceleration + df.movement_sprint_speed + df.movement_agility + df.movement_reactions + df.movement_balance)/5
   df['power_ratio'] = (df.power_jumping + df.power_shot_power + df.power_stamina + df.power_strength + df.power_long_shots) / 5
   df['mentality_ratio'] = (df.mentality_aggression + df.mentality_interceptions + 
                         df.mentality_positioning + df.mentality_vision + df.mentality_penalties + df.mentality_composure) / 6
   df['defending_ratio'] = (
      df.defending_marking_awareness + df.defending_standing_tackle + df.defending_sliding_tackle
      ) / 3
   df['gk_ratio'] = (
      df.goalkeeping_diving + df.goalkeeping_handling + df.goalkeeping_kicking + df.goalkeeping_positioning + df.goalkeeping_reflexes
      ) / 5
   df['attack_to_defense_ratio'] = (df.attacking_ratio / df.defending_ratio)
   df['skill_to_movement_ratio'] = (df.skill_ratio / df.movement_ratio)
   
   df['attack_to_movement_ratio'] = (df.attacking_ratio / df.movement_ratio)
   df['power_to_defence_ratio'] = (df.power_ratio/ df.defending_ratio)
   df['skill_to_defence_ratio'] = (df.skill_ratio / df.defending_ratio)
   df['attack_to_gk_ratio'] = (df.attacking_ratio / df.gk_ratio)
   
   for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(float, errors = 'ignore')
   
   return df

def EDA(df):
    total_na = df.isna().sum().sum()
    print('Dimensions:', df.shape[0], ' - rows', df.shape[1], ' - columns')
    print('Total NA\'s', total_na)
    print("%38s %10s     %10s %10s" % ("Column Name", "Data Type", "Count Distinct", "NA Values"))
    col_name = df.columns
    dtypes = df.dtypes
    uniq = df.nunique()
    na_val = df.isna().sum()
    for i in range(len(df.columns)):
        print("%38s %10s     %10s %10s" % (col_name[i], dtypes[i], uniq[i], na_val[i]))

def get_player_role(positions):
    if 'ST' in positions or 'CF' in positions:
        return 'Attack Player'
    elif 'CAM' in positions or 'LW' in positions or 'RW' in positions or 'RM' in positions or 'CM' in positions or 'LM' in positions:
        return 'Midfielder'
    elif 'RWB' in positions or 'CDM' in positions or 'LWB' in positions or 'LB' in positions or 'CB' in positions or 'RB' in positions:
        return 'Defender'
    elif 'GK' in positions:
        return 'Goalkeeper'
    else:
        return 'Unknown'

def oversample(X, Y):
    sm = SMOTE()
    X_resempled, Y_resempled = sm.fit_resample(X, Y)
    return X_resempled, Y_resempled

def scaling(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, scaler


In [76]:
df_check = pd.read_csv('fifa23_players.csv')
df_check = preprocess(df_check)

# duplicates_df = pd.DataFrame(columns=df_check.columns)  
# for index, row in df_check.iterrows():
#     positions = row['player_positions'].split(',')  
#     for position in positions:
#         duplicate = row.copy()
#         duplicate['player_positions'] = position.strip()
#         duplicates_df = duplicates_df.append(duplicate)

duplicates = []
for index, row in df_check.iterrows():
    positions = row['player_positions'].split(',')
    for position in positions:
        duplicate = row.copy()
        duplicate['player_positions'] = position.strip()
        duplicates.append(duplicate)

duplicates_df = pd.DataFrame(duplicates, columns=df_check.columns)
df_check = duplicates_df

del index, row, position, positions, duplicates_df, duplicate

encoder_target = LabelEncoder()
df_check.player_positions = encoder_target.fit_transform(df_check.player_positions)

df_check = df_check.astype(float, errors='ignore')

encoder_foot = LabelEncoder()
encoder_body = LabelEncoder()
encoder_work = LabelEncoder()
# categorical = df_check.select_dtypes(include='object').columns

df_check['work_rate'] = encoder_work.fit_transform(df_check['work_rate'])
df_check['body_type'] = encoder_body.fit_transform(df_check['body_type'])
df_check['preferred_foot'] = encoder_foot.fit_transform(df_check['preferred_foot'])

joblib.dump(encoder_work, 'Encoder_work.sav')
joblib.dump(encoder_body, 'Encoder_body.sav')
joblib.dump(encoder_foot, 'Encoder_foot.sav')

X = df_check.drop(columns='player_positions')
Y = df_check.player_positions

X_RES, Y_RES = oversample(X, Y)
# Оверсемплінг для усіх даних. Я памʼятаю на лекції Ви казали, що краще робити оверсемлпніг 
# тільки для трейн датасету, але, оскільки SMOTE не працює для мультитаргету, то мені потрбіно було створити дублікати гравців
# з різними позиціями, які у них були розділені через кому. 
# Чому дублікати? Тому що деякі гравці мають унікальні комбінації позицій, які представлені тільки в одному або двох екземплярах, 
# і тому SMOTE просто не може створити їх. Тому я семплю дані і потім усе повертаю на свої місця, звісно деякі засмплені дані втратяться, але ми 
# нагенеруємо 20000 нових записів і тим самим трохи збаланасуємо класи. 
# Якщо є певний спосіб зробити тільки для трейн сету семплінг, то я був би вдячний зf посилання на статтю, бо я не знайшов)

df_check = pd.concat([X_RES, Y_RES], axis=1)
df_check.player_positions = df_check.player_positions.astype(int)

df_check.player_positions = encoder_target.inverse_transform(df_check.player_positions)

# Групуємо дані за гравцями та об'єднуємо значення поля player_positions через кому
df_grouped = df_check.groupby('sofifa_id')['player_positions'].apply(', '.join).reset_index()

# Об'єднуємо згруповані дані з оригінальним фреймом за допомогою злиття
df_merged = pd.merge(df_check.drop(columns='player_positions'), df_grouped, on='sofifa_id')

# Видаляємо дублікати гравців, залишаючи тільки перший запис. Перевірка по айді гравця
df_final = df_merged.drop_duplicates(subset='sofifa_id')

# Скидуємо індекси рядків
df_check = df_final.reset_index(drop=True)

# Розділення позицій і застосування one-hot encoding
positions = df_check['player_positions'].str.split(', ')
one_hot = pd.get_dummies(positions.apply(pd.Series).stack()).sum(level=0)

# Об'єднання one-hot encoded даних з вихідним DataFrame
df_encoded = pd.concat([df_check, one_hot], axis=1)

del one_hot, positions, encoder_target, df_merged, df_grouped, df_final

df_encoded.drop(columns=['sofifa_id', 'player_positions'], inplace=True, errors='ignore')
scaler_x = StandardScaler()

columns = df_encoded.columns[:-15]

# X = df_encoded.drop(columns=columns)
# columns = df_encoded[columns]
X = df_encoded[columns]
Y = df_encoded.iloc[:, -15:]
# scaling and loading
X = scaler_x.fit_transform(X)

joblib.dump(scaler_x, 'Scaler.sav')

X = pd.DataFrame(X, columns = columns)

X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=0.8, random_state=0)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=0)

x_columns = X.columns

Index(['player_url', 'short_name', 'long_name', 'player_positions', 'dob',
       'club_position', 'club_name', 'league_name', 'club_loaned_from',
       'club_joined', 'nationality_name', 'nation_position', 'preferred_foot',
       'work_rate', 'body_type', 'player_tags', 'player_traits',
       'player_face_url', 'club_logo_url', 'nation_logo_url',
       'nation_flag_url'],
      dtype='object')

In [48]:
df_d = pd.read_csv('fifa23_players.csv')
df_d = preprocess(df_d)
columns_d = df_d.columns
# duplicates_df = pd.DataFrame(columns=df_check.columns)  
# for index, row in df_check.iterrows():
#     positions = row['player_positions'].split(',')  
#     for position in positions:
#         duplicate = row.copy()
#         duplicate['player_positions'] = position.strip()
#         duplicates_df = duplicates_df.append(duplicate)

# df_d = duplicates_df
duplicates = []
for index, row in df_check.iterrows():
    positions = row['player_positions'].split(',')
    for position in positions:
        duplicate = row.copy()
        duplicate['player_positions'] = position.strip()
        duplicates.append(duplicate)

duplicates_df = pd.DataFrame(duplicates, columns=df_check.columns)
df_d = duplicates_df

del index, row, position, positions, duplicates_df, duplicate

encoder_target_d = LabelEncoder()
df_d.player_positions = encoder_target_d.fit_transform(df_d.player_positions)

df_d = df_d.astype(float, errors='ignore')

encoder_d = LabelEncoder()
# categorical = df_d.select_dtypes(include='object').columns

df_d['work_rate'] = encoder_work.transform(df_d['work_rate'])
df_d['body_type'] = encoder_body.transform(df_d['body_type'])
df_d['preferred_foot'] = encoder_foot.transform(df_d['preferred_foot'])

# joblib.dump(encoder_work, 'Encoder_work.sav')
# joblib.dump(encoder_body, 'Encoder_body.sav')
# joblib.dump(encoder_foot, 'Encoder_foot.sav')

# df_d[categorical] = df_d[categorical].apply(encoder.fit_transform)

# X_D = df_d.drop(columns='player_positions')
# Y_D = df_d.player_positions

#drop correlated data for Linear Models
df_to_split = df_d.drop(columns=['shooting', 'dribbling', 'physic', 'attacking_finishing', 'attacking_short_passing', 'skill_dribbling', 
    'skikk_fk_accuracy', 'skill_ball_contol', 'movement_acceleration',
    'mentality_agility','power_long_shots', 'mentality_interceptions',
    'mentality_positioning', 'defending_marking_awareness', 'defending_sliding_tackle',
    'goalkeeping_handling', 'goalkeeping_positioning', 'height_to_weight_ratio', 
    'value_to_wage_ratio'], errors='ignore')

X_train_d, Y_train_d, X_valid_d, Y_valid_d, X_test_d, Y_test_d = train_valid_test_split(df_to_split, target='player_positions', train_size=0.8, valid_size=0.1, test_size=0.1, random_state=42)

X_train_sampled_d, Y_train_sampled_d = oversample(X_train_d, Y_train_d)

# X_train_sampled_d = scaling(X_train_sampled_d)


In [82]:
def logreg_pipeline(X_train, Y_train):
    model = LogisticRegression(solver='newton-cg', multi_class='multinomial', max_iter=500)
    scaler = StandardScaler()
    logreg_pipeline = make_pipeline(scaler, model)
    logreg_pipeline.fit(X_train, Y_train)
    return logreg_pipeline

def logregcv_pipeline(X_train, Y_train, cv = 3, cs = 10):
    scaler = StandardScaler()
    model = LogisticRegressionCV(max_iter=1000, Cs=cs, cv=cv, solver='newton-cg', multi_class='multinomial')
    logregcv_pipeline = make_pipeline(scaler, model)
    logregcv_pipeline.fit(X_train, Y_train)
    return logregcv_pipeline

def forest_pipeline(X_train, Y_train, n_estimators=200):
    model = RandomForestClassifier(n_estimators=n_estimators)
    scaler = StandardScaler()
    pipeline = make_pipeline(scaler, model)
    pipeline.fit(X_train, Y_train)
    return pipeline


def sgd_pipeline(X_train, Y_train):
    sgd = SGDClassifier(max_iter = 500)
    scaler = StandardScaler()
    sgd_pipeline = make_pipeline(scaler, CalibratedClassifierCV(sgd)) # gradient vanishing, gradient explosion
    sgd_pipeline.fit(X_train, Y_train)
    return sgd_pipeline

def pipeline(model, X_train,Y_train):
    scaler = StandardScaler()
    pipeline = make_pipeline(scaler,
                             model())
    pipeline.fit(X_train, Y_train)
    return pipeline

def play(pipeline, X_test, scale = False):
    if scale:
        X_test = StandardScaler().fit_transform(X_test)
    predictions = pipeline.predict(X_test)
    return pipeline, predictions

def store_pipeline(pipeline, name):
    pipeline_file = open(name + '.sav', 'wb')
    joblib.dump(pipeline, pipeline_file)
    pipeline_file.close
    
def neural_pipeline(X_train, Y_train, steps = 300, batch_size = 300, epochs = 100, split = 0.2, scale = False): 
    if scale:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
    k_neurons = X_train.shape[1]
    k_output = Y_train.shape[1]
    
    neural = tf.keras.Sequential()
    stop = EarlyStopping()
    input  = tf.keras.layers.Dense(k_neurons, input_shape = [k_neurons], activation='elu')
    neural.add(input)

    h1 = tf.keras.layers.Dense(k_neurons*4, activation='tanh')
    neural.add(h1)
    h2 = tf.keras.layers.Dense(k_neurons*8, activation='tanh')
    neural.add(h2)
    h3 = tf.keras.layers.Dense(k_neurons*16, activation='tanh')
    neural.add(h3)
    h4 = tf.keras.layers.Dense(k_neurons*32, activation='tanh')
    neural.add(h4)
    # h5 = tf.keras.layers.Dense(k_neurons*64, activation='tanh')
    # neural.add(h5)
    out = tf.keras.layers.Dense(k_output, activation='softmax')
    neural.add(out)

    neural.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = neural.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=split, steps_per_epoch=steps,callbacks=[stop])
    
    return history, neural

def play_neural(pipeline, X, Y, scale = False):
    if scale:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    loss, accuracy = pipeline.evaluate(X, Y)
    return loss, accuracy

def get_accuracy(pred, y):
    return accuracy_score(y, pred)

In [ ]:
logreg = logreg_pipeline(X_train_sampled_d, Y_train_sampled_d)


In [ ]:
store_pipeline(logreg, 'logistic')
p, predictions = play(logreg, X_test_d)

In [15]:
forest_pipeline_one_hot = forest_pipeline(X_train, Y_train)
p, predictions = play(forest_pipeline_one_hot, X_valid)


In [17]:
store_pipeline(pipeline=forest_pipeline_one_hot, name='RandomForest_for_One_Hot_Encoded_Taget')

In [19]:
# accuracy_score(Y_test_d, predictions)
p, predictions = play(logreg, StandardScaler().fit_transform(X_valid_d))
accuracy_score(Y_valid_d, predictions)

0.4673878460069997

In [24]:
history, neural_p = neural_pipeline(X_train=X_train, Y_train=Y_train, batch_size=300, scale=True, steps=300)

Epoch 1/100
300/300 [==============================] - 7s 21ms/step - loss: 0.1718 - accuracy: 0.4917 - val_loss: 0.1531 - val_accuracy: 0.5360
Epoch 2/100
300/300 [==============================] - 6s 21ms/step - loss: 0.1545 - accuracy: 0.5239 - val_loss: 0.1513 - val_accuracy: 0.5182
Epoch 3/100
300/300 [==============================] - 6s 21ms/step - loss: 0.1483 - accuracy: 0.5440 - val_loss: 0.1445 - val_accuracy: 0.5664
Epoch 4/100
300/300 [==============================] - 6s 21ms/step - loss: 0.1406 - accuracy: 0.5729 - val_loss: 0.1424 - val_accuracy: 0.5704
Epoch 5/100
300/300 [==============================] - 6s 21ms/step - loss: 0.1288 - accuracy: 0.6175 - val_loss: 0.1331 - val_accuracy: 0.6193
Epoch 6/100
300/300 [==============================] - 6s 22ms/step - loss: 0.1132 - accuracy: 0.6729 - val_loss: 0.1280 - val_accuracy: 0.6463
Epoch 7/100
300/300 [==============================] - 7s 22ms/step - loss: 0.0955 - accuracy: 0.7293 - val_loss: 0.1228 - val_accuracy:

In [25]:
loss, accuracy = play_neural(neural_p, X_valid, Y_valid, scale=True)
print(loss, accuracy)

157/157 [==============================] - 0s 2ms/step - loss: 0.1261 - accuracy: 0.6831
0.12614162266254425 0.6830677390098572


In [42]:
forest_pipeline_for_duplicates = forest_pipeline(X_train_sampled_d, Y_train_sampled_d)
p, predictions = play(forest_pipeline_for_duplicates, X_test_d)

In [40]:
get_accuracy(predictions, Y_test_d)

0.48035629075870845

In [7]:
store_pipeline(neural_p, 'neural_pipeline')

In [43]:
store_pipeline(forest_pipeline_for_duplicates, 'RandomForest_for_duplicated_Taget')

In [83]:
dd = pd.read_csv('fifa23_players.csv')
# dd = preprocess(dd)

urls = [idx for idx, col in enumerate(dd.columns) if 'url' in col]
urls = [dd.columns[idx] for idx in urls]
dd = dd.drop(columns= urls)
columns_d = dd.columns
duplicates_df = pd.DataFrame(columns=dd.columns)  
for index, row in dd.iterrows():
    positions = row['player_positions'].split(',')  
    for position in positions:
        duplicate = row.copy()
        duplicate['player_positions'] = position.strip()
        duplicates_df = duplicates_df.append(duplicate)

dd = duplicates_df


In [84]:
dd = dd.astype(int, errors='ignore')

In [88]:
dd_ = pd.read_csv('fifa23_players.csv')
urls = [idx for idx, col in enumerate(dd_.columns) if 'url' in col]
urls = [dd_.columns[idx] for idx in urls]
dd_ = dd_.drop(columns= urls)

In [66]:
def top_100_overall(df):
    top_100_overall = df.nlargest(100, 'overall')
    return top_100_overall

top_100_overall(dd_)



,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,...,96,20,35,24,6,11,15,14,8,NaN
1,188545,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,...,88,35,42,19,15,6,12,8,10,NaN
2,20801,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,...,95,24,32,24,7,11,15,14,11,NaN
3,190871,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,1992-02-05,...,93,35,32,29,9,9,15,15,11,NaN
4,192985,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,1991-06-28,...,89,68,65,53,15,13,5,10,13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,235212,A. Hakimi,Achraf Hakimi Mouh,"RB, RWB",85,88,69500000.0,100000.0,22,1998-11-04,...,80,75,78,76,10,8,14,6,8,NaN
96,235243,M. de Ligt,Matthijs de Ligt,CB,85,90,75000000.0,81000.0,21,1999-08-12,...,82,84,85,85,12,11,11,12,10,NaN
97,41236,Z. Ibrahimović,Zlatan Ibrahimović,ST,84,84,14500000.0,51000.0,39,1981-10-03,...,90,28,37,24,13,15,10,9,12,NaN
98,146536,Jesús Navas,Jesús Navas González,"RB, RM",84,84,13000000.0,30000.0,35,1985-11-21,...,82,82,77,79,10,10,15,7,13,NaN


In [67]:
def top_100_wage(df):
    top_100_wage = df.nlargest(100, 'wage_eur')
    return top_100_wage

top_100_wage(dd_)

,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
4,192985,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,1991-06-28,...,89,68,65,53,15,13,5,10,13,NaN
11,165153,K. Benzema,Karim Benzema,"CF, ST",89,89,66000000.0,350000.0,33,1987-12-19,...,90,43,24,18,13,11,5,5,7,NaN
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,...,96,20,35,24,6,11,15,14,8,NaN
14,200145,Casemiro,Carlos Henrique Venancio Casimiro,CDM,89,89,88000000.0,310000.0,29,1992-02-23,...,84,85,88,87,13,14,16,12,12,NaN
24,182521,T. Kroos,Toni Kroos,CM,88,88,75000000.0,310000.0,31,1990-01-04,...,88,71,73,60,10,11,13,7,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,240130,Éder Militão,Éder Gabriel Militão,CB,82,89,56500000.0,130000.0,23,1998-01-18,...,74,83,83,82,11,8,15,12,15,NaN
342,211300,A. Martial,Anthony Martial,"ST, LM",81,84,34500000.0,130000.0,25,1995-12-05,...,78,26,33,31,9,8,8,15,11,NaN
631,232432,L. Jović,Luka Jović,ST,79,84,27000000.0,130000.0,23,1997-12-23,...,78,25,26,16,8,7,12,9,13,NaN
31,167948,H. Lloris,Hugo Lloris,GK,87,87,13500000.0,125000.0,34,1986-12-26,...,65,29,10,18,88,83,65,84,90,61.0


In [68]:
def top_30_gk(df):
    top_30_gk = df[df['player_positions'] == 'GK'].nlargest(30, 'overall')
    return top_30_gk

top_30_gk(dd_)

,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
5,200389,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,...,68,27,12,18,87,92,78,90,90,50.0
7,167495,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,...,70,17,10,11,88,88,91,89,88,56.0
8,192448,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,...,70,25,13,10,88,85,88,88,90,43.0
12,192119,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,...,66,20,18,16,84,89,74,86,88,46.0
18,210257,Ederson,Ederson Santana de Moraes,GK,89,91,94000000.0,200000.0,27,1993-08-17,...,70,29,15,8,87,82,93,88,88,64.0
20,212831,Alisson,Alisson Ramsés Becker,GK,89,90,82000000.0,190000.0,28,1992-10-02,...,65,15,19,16,86,86,84,90,89,52.0
21,230621,G. Donnarumma,Gianluigi Donnarumma,GK,89,93,119500000.0,110000.0,22,1999-02-25,...,68,20,14,16,91,83,79,85,90,52.0
26,193041,K. Navas,Keylor Navas Gamboa,GK,88,88,15500000.0,130000.0,34,1986-12-15,...,67,28,14,14,89,84,75,87,89,54.0
31,167948,H. Lloris,Hugo Lloris,GK,87,87,13500000.0,125000.0,34,1986-12-26,...,65,29,10,18,88,83,65,84,90,61.0
34,186153,W. Szczęsny,Wojciech Tomasz Szczęsny,GK,87,87,42000000.0,105000.0,31,1990-04-18,...,65,20,13,12,86,82,73,87,88,49.0


In [86]:
def top_30_clubs_by_player_overall(df):
    top_30_clubs_by_player_overall = df.groupby('club_name')['overall'].max().nlargest(30)
    return top_30_clubs_by_player_overall

def top_30_clubs_sum_players_overall(df):
    top_30_clubs_sum_players_overall = df.groupby('club_name')['overall'].mean().nlargest(30)
    return top_30_clubs_sum_players_overall

def top_30_clubs_avg_pace(df):
    top_30_clubs_avg_pace = df.groupby('club_name')['pace'].mean().nlargest(30)
    return top_30_clubs_avg_pace

def top_leagues_dribbling(df):
    top_leagues_dribbling = df.groupby('league_name')['dribbling'].mean().sort_values(ascending=False)
    return top_leagues_dribbling

def top_teams(df):
    # Reset the index to make 'club_name' a regular column
    df_reset = df.reset_index()

    # Filter players by positions and group by 'club_name'
    goalkeepers = df_reset[df_reset['player_positions'] == 'GK'].groupby('club_name').apply(lambda x: x.nlargest(1, 'overall')).reset_index(drop=True)
    defenders = df_reset[df_reset['player_positions'].str.contains('RWB|CDM|LWB|LB|CB|RB')].groupby('club_name').apply(lambda x: x.nlargest(4, 'overall')).reset_index(drop=True)
    midfielders = df_reset[df_reset['player_positions'].str.contains('CAM|LW|RW|RM|CM|LM')].groupby('club_name').apply(lambda x: x.nlargest(4, 'overall')).reset_index(drop=True)
    strikers = df_reset[df_reset['player_positions'].str.contains('ST|CF')].groupby('club_name').apply(lambda x: x.nlargest(2, 'overall')).reset_index(drop=True)

    # Concatenate filtered data
    top_players = pd.concat([goalkeepers, defenders, midfielders, strikers])

    # Group by 'club_name' and compute the overall rating sum
    top_teams = top_players.groupby('club_name')['overall'].sum().nlargest(30)
    return top_teams

dd['player_role'] = dd['player_positions'].apply(get_player_role)

def distribution_by_role_age(df):
    positions = ['CF', 'ST', 'RW', 'LW', 'CAM', 'RM', 'CM', 'LM', 'RWB', 'CDM', 'LWB', 'LB', 'CB', 'RB', 'GK']
    filtered_data = df[df['player_role'] != 'Unknown']


    plt.figure(figsize=(12, 6))
    sns.boxplot(x='player_role', y='age', data=filtered_data)
    plt.xlabel('Роль гравця')
    plt.ylabel('Вік')
    plt.title('Розподіл віку гравців за ролями')
    plt.xticks(rotation=45)
    plt.show()

def distribution_by_age_pos(df):
    data = df[df.player_positions.isin(positions)]
    plt.figure(figsize=(12,6))
    sns.boxenplot(data = data, x = 'player_positions', y = 'age')
    plt.xlabel('Позиція')
    plt.ylabel('Вік')
    plt.title('Розподіл віку гравців за позиціями')
    plt.xticks(rotation=45)
    plt.show()

def distribution_by_overall_pos(df):
    data = df[df.player_positions.isin(positions)]

    plt.figure(figsize=(12,6))
    sns.boxenplot(data = data, x = 'player_positions', y = 'overall')
    plt.xlabel('Позиція')
    plt.ylabel('Рейтинг')
    plt.title('Розподіл рейтингу гравців за позиціями')
    plt.xticks(rotation=45)
    plt.show()

def dis_by_nationalities(df):
    nationalities = [nation for nation, count in df['nationality_name'].value_counts().items() if count > 210]
    len(nationalities)
    data = df[df['nationality_name'].isin(nationalities)]
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=data, x='nationality_name', y='overall')
    plt.xlabel('Національність')
    plt.ylabel('Рейтинг')
    plt.title('Розподіл рейтингу гравців за національністю')
    plt.xticks(rotation=90)
    plt.show()

def penalty_score(df):
    
    data = df
    data['Penaltyist_Score'] = (0.4 * df['mentality_penalties'] + 0.4 * df['shooting'] + 0.1 * df['power_shot_power'] + 0.1 * df['mentality_vision']) / 4
    top_clubs = df.groupby('club_name')['Penaltyist_Score'].mean().nlargest(10)
    return top_clubs

def penalty_score_gk(df):
    data = df
    data['GK_penalty_score'] = (0.3 * data.goalkeeping_diving + 0.1 * data.goalkeeping_handling + 0.3 * data.goalkeeping_reflexes + 0.2 * data.goalkeeping_speed + 0.1 * data.mentality_penalties) / 5
    top_clubs = df.groupby('club_name')['GK_penalty_score'].mean().nlargest(10)
    return top_clubs

